In [30]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [31]:
# Importing data
movies = pd.read_csv("movies.dat", sep = "::", header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv("ratings.dat", sep = "::", header=None, engine='python', encoding='latin-1')
users = pd.read_csv("users.dat", sep = "::", header=None, engine='python', encoding='latin-1')

In [39]:
# Importing the training and testing set
xtrain = pd.read_csv("training_set.csv")
xtest = pd.read_csv("test_set.csv")

# Converting the training and testing set into arrays
xtrain = np.array(xtrain, dtype = 'int')
xtest = np.array(xtest, dtype='int')

In [40]:
xtrain.shape

(750121, 4)

In [41]:
# Counting the number of users and movies
nb_users = int(max(max(xtrain[:,0]), max(xtest[:,0])))
nb_movies = int(max(max(xtrain[:,1]), max(xtest[:,1])))
print(nb_users, nb_movies)

6040 3952


In [42]:
# Converting users and movies into matrix with users in rows and movies in column

def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
xtrain = convert(xtrain)
xtest = convert(xtest)

In [44]:
# Converting training and test data into tensors
xtrain = torch.FloatTensor(xtrain)
xtest = torch.FloatTensor(xtest)

In [45]:
xtrain.shape

torch.Size([6040, 3952])

In [46]:
# Creating the Autoencoder class

class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies, 20)
    self.fc2 = nn.Linear(20,10)
    self.fc3 = nn.Linear(10,20)
    self.fc4 = nn.Linear(20, nb_movies)
    self.activation = nn.Sigmoid()
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x

In [47]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [52]:
# Training the model

n_epochs = 200
for epoch in range(1, n_epochs + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(xtrain[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+' loss: '+ str(train_loss/s))

epoch: 1 loss: tensor(0.8863)
epoch: 2 loss: tensor(0.8861)
epoch: 3 loss: tensor(0.8806)
epoch: 4 loss: tensor(0.8785)
epoch: 5 loss: tensor(0.8770)
epoch: 6 loss: tensor(0.8767)
epoch: 7 loss: tensor(0.8753)
epoch: 8 loss: tensor(0.8750)
epoch: 9 loss: tensor(0.8750)
epoch: 10 loss: tensor(0.8747)


KeyboardInterrupt: ignored

In [51]:
# Testing the model

test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(xtrain[id_user]).unsqueeze(0)
  target = Variable(xtest[id_user]).unsqueeze(0)
  if (torch.sum(target.data > 0) > 0):
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    train_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.    
  print(Loss: '+ str(train_loss/s))

Streaming output truncated to the last 5000 lines.
epoch: 200 loss: tensor(6.0195)
epoch: 200 loss: tensor(6.0148)
epoch: 200 loss: tensor(6.0102)
epoch: 200 loss: tensor(6.0052)
epoch: 200 loss: tensor(6.0007)
epoch: 200 loss: tensor(5.9960)
epoch: 200 loss: tensor(5.9910)
epoch: 200 loss: tensor(5.9856)
epoch: 200 loss: tensor(5.9808)
epoch: 200 loss: tensor(5.9758)
epoch: 200 loss: tensor(5.9713)
epoch: 200 loss: tensor(5.9665)
epoch: 200 loss: tensor(5.9616)
epoch: 200 loss: tensor(5.9568)
epoch: 200 loss: tensor(5.9520)
epoch: 200 loss: tensor(5.9476)
epoch: 200 loss: tensor(5.9427)
epoch: 200 loss: tensor(5.9378)
epoch: 200 loss: tensor(5.9332)
epoch: 200 loss: tensor(5.9284)
epoch: 200 loss: tensor(5.9238)
epoch: 200 loss: tensor(5.9189)
epoch: 200 loss: tensor(5.9142)
epoch: 200 loss: tensor(5.9093)
epoch: 200 loss: tensor(5.9044)
epoch: 200 loss: tensor(5.8998)
epoch: 200 loss: tensor(5.8948)
epoch: 200 loss: tensor(5.8901)
epoch: 200 loss: tensor(5.8859)
epoch: 200 loss: tens